# Курсовой проект

## Курс "Теория вероятностей и математическая статистика"

### "Статистическое сопоставление симптомов у пациентов, имеющих рак легких и определение тех, которые встречаются достоверно"

In [1]:
# импортируем необходимые для расчета библиотеки
import pandas as pd
import numpy as np
from math import sqrt
import seaborn as sns
import matplotlib.pyplot as plt
import scipy as sp
import scipy.stats
from scipy.stats import fisher_exact

import warnings
warnings.filterwarnings('ignore')

### 1. Download dataset

In [2]:
data_set = 'lung_cancer.csv'

In [3]:
data = pd.read_csv(data_set, sep=',')
data.head()

,GENDER,AGE,SMOKING,YELLOW_FINGERS,ANXIETY,PEER_PRESSURE,CHRONIC_DISEASE,FATIGUE,ALLERGY,WHEEZING,ALCOHOL_CONSUMING,COUGHING,SHORTNESS_OF_BREATH,SWALLOWING_DIFFICULTY,CHEST_PAIN,LUNG_CANCER
0,M,69,1,2,2,1,1,2,1,2,2,2,2,2,2,YES
1,M,74,2,1,1,1,2,2,2,1,1,1,2,2,2,YES
2,F,59,1,1,1,2,1,2,1,2,1,2,2,1,2,NO
3,M,63,2,2,2,1,1,1,1,1,2,1,1,2,2,NO
4,F,63,1,2,1,1,1,1,1,2,1,2,2,1,1,NO


In [4]:
data.tail()

,GENDER,AGE,SMOKING,YELLOW_FINGERS,ANXIETY,PEER_PRESSURE,CHRONIC_DISEASE,FATIGUE,ALLERGY,WHEEZING,ALCOHOL_CONSUMING,COUGHING,SHORTNESS_OF_BREATH,SWALLOWING_DIFFICULTY,CHEST_PAIN,LUNG_CANCER
304,F,56,1,1,1,2,2,2,1,1,2,2,2,2,1,YES
305,M,70,2,1,1,1,1,2,2,2,2,2,2,1,2,YES
306,M,58,2,1,1,1,1,1,2,2,2,2,1,1,2,YES
307,M,67,2,1,2,1,1,2,2,1,2,2,2,1,2,YES
308,M,62,1,1,1,2,1,2,2,2,2,1,1,2,1,YES


In [5]:
# всего в датасете 309 записей

##### посмотрим типы данных столбцов (признаков)

In [6]:
data.dtypes

GENDER                   object
AGE                       int64
SMOKING                   int64
YELLOW_FINGERS            int64
ANXIETY                   int64
PEER_PRESSURE             int64
CHRONIC_DISEASE           int64
FATIGUE                   int64
ALLERGY                   int64
WHEEZING                  int64
ALCOHOL_CONSUMING         int64
COUGHING                  int64
SHORTNESS_OF_BREATH       int64
SWALLOWING_DIFFICULTY     int64
CHEST_PAIN                int64
LUNG_CANCER              object
dtype: object

In [7]:
# Attribute information / Информация об атрибутах:

# Gender: M(male), F(female) - пол
# Age: Age of the patient - возраст 
# Smoking: YES=2 , NO=1. - курение 
# Yellow fingers: YES=2 , NO=1. - желтые пальцы
# Anxiety: YES=2 , NO=1. - тревога
# Peer_pressure: YES=2 , NO=1. - скачки давления 
# Chronic Disease: YES=2 , NO=1. - хроническое заболевание 
# Fatigue: YES=2 , NO=1. - усталость
# Allergy: YES=2 , NO=1. - аллергия 
# Wheezing: YES=2 , NO=1. - свистящее дыхание 
# Alcohol: YES=2 , NO=1. - употребление алкоголя 
# Coughing: YES=2 , NO=1. - кашель 
# Shortness of Breath: YES=2 , NO=1. - одышка 
# Swallowing Difficulty: YES=2 , NO=1. - затрудненное глотание 
# Chest pain: YES=2 , NO=1. - боль в груди
# Lung Cancer: YES , NO. - рак легких 

### Этапы проверки:
#### 1. Сформулировать основную и альтернативную гипотезы.
##### - основная гипотеза, что симптомы: "желтые пальцы", "тревога", "скачки давления", "усталость", "аллергия", "свистящее дыхание", "кашель", "одышка", "затрудненное глотание", "боли в груди" чаще встречаются у больных с раком легких
#### 2. Выбрать статистический критерий, по которому будет проводиться проверка.
##### - критерий Фишера 
#### 3. Задать уровень значимости α или вероятность, с которой будет выполняться эта гипотеза.
#####  α = 0,05
#### 4. Определить границы области гипотезы.
#### 5. Подвести итоги и сформулировать вывод.

## 2. EDA (exploratory data analysis)

In [8]:
data.describe()

,AGE,SMOKING,YELLOW_FINGERS,ANXIETY,PEER_PRESSURE,CHRONIC_DISEASE,FATIGUE,ALLERGY,WHEEZING,ALCOHOL_CONSUMING,COUGHING,SHORTNESS_OF_BREATH,SWALLOWING_DIFFICULTY,CHEST_PAIN
count,309.000000,309.000000,309.000000,309.000000,309.000000,309.000000,309.000000,309.000000,309.000000,309.000000,309.000000,309.000000,309.000000,309.000000
mean,62.673139,1.563107,1.569579,1.498382,1.501618,1.504854,1.673139,1.556634,1.556634,1.556634,1.579288,1.640777,1.469256,1.556634
std,8.210301,0.496806,0.495938,0.500808,0.500808,0.500787,0.469827,0.497588,0.497588,0.497588,0.494474,0.480551,0.499863,0.497588
min,21.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
25%,57.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
50%,62.000000,2.000000,2.000000,1.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,1.000000,2.000000
75%,69.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000
max,87.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000


##### Кодирование Признаков (Encoding Categorical Features)¶

In [9]:
# в датасете имеются два столбца - GENDER и LUNG_CANCER c с типом данным 'object'
# закодируем их:

In [10]:
cleanup_nums_GENDER = {"GENDER":  {"F": 1, "M": 2}, "LUNG_CANCER": {"YES": 2, "NO": 1}}

In [11]:
data[['GENDER', 'LUNG_CANCER']] = data[['GENDER', 'LUNG_CANCER']].replace(cleanup_nums_GENDER)

In [12]:
data.head()

,GENDER,AGE,SMOKING,YELLOW_FINGERS,ANXIETY,PEER_PRESSURE,CHRONIC_DISEASE,FATIGUE,ALLERGY,WHEEZING,ALCOHOL_CONSUMING,COUGHING,SHORTNESS_OF_BREATH,SWALLOWING_DIFFICULTY,CHEST_PAIN,LUNG_CANCER
0,2,69,1,2,2,1,1,2,1,2,2,2,2,2,2,2
1,2,74,2,1,1,1,2,2,2,1,1,1,2,2,2,2
2,1,59,1,1,1,2,1,2,1,2,1,2,2,1,2,1
3,2,63,2,2,2,1,1,1,1,1,2,1,1,2,2,1
4,1,63,1,2,1,1,1,1,1,2,1,2,2,1,1,1


In [13]:
# в качестве симптомов выступают следующие признаки:
YELLOW_FINGERS = data['YELLOW_FINGERS'] # желтые пальцы 
ANXIETY = data['ANXIETY'] # тревога
PEER_PRESSURE = data['PEER_PRESSURE'] # скачки давления 
FATIGUE = data['FATIGUE'] # усталость 
ALLERGY = data['ALLERGY'] # аллергия
WHEEZING = data['WHEEZING'] # свистящее дыхание 
COUGHING = data['COUGHING'] # кашель
SHORTNESS_OF_BREATH = data['SHORTNESS_OF_BREATH'] # одышка 
SWALLOWING_DIFFICULTY = data['SWALLOWING_DIFFICULTY'] # затрудненное глотание
CHEST_PAIN = data['CHEST_PAIN'] # боли в груди 

In [14]:
data.groupby('LUNG_CANCER')['YELLOW_FINGERS'].value_counts()

LUNG_CANCER  YELLOW_FINGERS
1            1                  26
             2                  13
2            2                 163
             1                 107
Name: YELLOW_FINGERS, dtype: int64

обычно уровень значимости принимается как 5% (или 0,05), то меньше 5 (0,05) - различия достоверны

In [15]:
YELLOW_FINGERS = [[26, 107],[13, 163]]
oddsratio, pvalue = fisher_exact(YELLOW_FINGERS, alternative='two-sided')
print(f'несбалансированное соотношениие составляет: {pvalue} или {round(pvalue * 100, 2)}%')
if pvalue > 0.05:
    print(f'статистически незначимо, так как {pvalue} > 0.05')
else:
    print(f'статистически значимо, так как {pvalue} <= 0.05')
    print(f'симптом "желтые пальцы" достоверно чаще встречается у больных раком легких')

несбалансированное соотношениие составляет: 0.0017475195910311545 или 0.17%
статистически значимо, так как 0.0017475195910311545 <= 0.05
симптом "желтые пальцы" достоверно чаще встречается у больных раком легких


In [16]:
data.groupby('LUNG_CANCER')['ANXIETY'].value_counts()

LUNG_CANCER  ANXIETY
1            1           27
             2           12
2            2          142
             1          128
Name: ANXIETY, dtype: int64

In [17]:
ANXIETY = [[27, 128],[12, 142]]
oddsratio, pvalue = fisher_exact(ANXIETY, alternative='two-sided')
print(f'несбалансированное соотношениие составляет: {pvalue} или {round(pvalue * 100, 2)}%')
if pvalue > 0.05:
    print(f'статистически незначимо, так как {pvalue} > 0.05')
else:
    print(f'статистически значимо, так как {pvalue} <= 0.05')
    print(f'симптом "тревога" достоверно чаще встречается у больных раком легких')

несбалансированное соотношениие составляет: 0.015643534176720708 или 1.56%
статистически значимо, так как 0.015643534176720708 <= 0.05
симптом "тревога" достоверно чаще встречается у больных раком легких


In [18]:
data.groupby('LUNG_CANCER')['PEER_PRESSURE'].value_counts()

LUNG_CANCER  PEER_PRESSURE
1            1                 29
             2                 10
2            2                145
             1                125
Name: PEER_PRESSURE, dtype: int64

In [19]:
ANXIETY = [[29, 125],[10, 145]]
oddsratio, pvalue = fisher_exact(ANXIETY, alternative='two-sided')
print(f'несбалансированное соотношениие составляет: {pvalue} или {round(pvalue * 100, 2)}%')
if pvalue > 0.05:
    print(f'статистически незначимо, так как {pvalue} > 0.05')
else:
    print(f'статистически значимо, так как {pvalue} <= 0.05')
    print(f'симптом "скачки давления" достоверно чаще встречается у больных раком легких')

несбалансированное соотношениие составляет: 0.0010713435469612903 или 0.11%
статистически значимо, так как 0.0010713435469612903 <= 0.05
симптом "скачки давления" достоверно чаще встречается у больных раком легких


In [20]:
data.groupby('LUNG_CANCER')['FATIGUE'].value_counts()

LUNG_CANCER  FATIGUE
1            1           20
             2           19
2            2          189
             1           81
Name: FATIGUE, dtype: int64

In [21]:
FATIGUE = [[20, 81],[19, 189]]
oddsratio, pvalue = fisher_exact(FATIGUE, alternative='two-sided')
print(f'несбалансированное соотношениие составляет: {pvalue} или {round(pvalue * 100, 2)}%')
if pvalue > 0.05:
    print(f'статистически незначимо, так как {pvalue} > 0.05')
else:
    print(f'статистически значимо, так как {pvalue} <= 0.05')
    print(f'симптом "усталость" достоверно чаще встречается у больных раком легких')

несбалансированное соотношениие составляет: 0.010521933794852556 или 1.05%
статистически значимо, так как 0.010521933794852556 <= 0.05
симптом "усталость" достоверно чаще встречается у больных раком легких


In [22]:
data.groupby('LUNG_CANCER')['ALLERGY'].value_counts()

LUNG_CANCER  ALLERGY
1            1           34
             2            5
2            2          167
             1          103
Name: ALLERGY, dtype: int64

In [23]:
ALLERGY = [[34, 103],[5, 167]]
oddsratio, pvalue = fisher_exact(ALLERGY, alternative='two-sided')
print(f'несбалансированное соотношениие составляет: {pvalue} или {round(pvalue * 100, 2)}%')
if pvalue > 0.05:
    print(f'статистически незначимо, так как {pvalue} > 0.05')
else:
    print(f'статистически значимо, так как {pvalue} <= 0.05')
    print(f'симптом "аллергия" достоверно чаще встречается у больных раком легких')

несбалансированное соотношениие составляет: 4.873476581685877e-09 или 0.0%
статистически значимо, так как 4.873476581685877e-09 <= 0.05
симптом "аллергия" достоверно чаще встречается у больных раком легких


In [24]:
data.groupby('LUNG_CANCER')['WHEEZING'].value_counts()

LUNG_CANCER  WHEEZING
1            1            30
             2             9
2            2           163
             1           107
Name: WHEEZING, dtype: int64

In [25]:
WHEEZING = [[30, 107],[9, 163]]
oddsratio, pvalue = fisher_exact(WHEEZING, alternative='two-sided')
print(f'несбалансированное соотношениие составляет: {pvalue} или {round(pvalue * 100, 2)}%')
if pvalue > 0.05:
    print(f'статистически незначимо, так как {pvalue} > 0.05')
else:
    print(f'статистически значимо, так как {pvalue} <= 0.05')
    print(f'симптом "свистящее дыхание" достоверно чаще встречается у больных раком легких')

несбалансированное соотношениие составляет: 1.9623161586515003e-05 или 0.0%
статистически значимо, так как 1.9623161586515003e-05 <= 0.05
симптом "свистящее дыхание" достоверно чаще встречается у больных раком легких


In [26]:
data.groupby('LUNG_CANCER')['COUGHING'].value_counts()

LUNG_CANCER  COUGHING
1            1            29
             2            10
2            2           169
             1           101
Name: COUGHING, dtype: int64

In [27]:
COUGHING = [[29, 101],[10, 169]]
oddsratio, pvalue = fisher_exact(COUGHING, alternative='two-sided')
print(f'несбалансированное соотношениие составляет: {pvalue} или {round(pvalue * 100, 2)}%')
if pvalue > 0.05:
    print(f'статистически незначимо, так как {pvalue} > 0.05')
else:
    print(f'статистически значимо, так как {pvalue} <= 0.05')
    print(f'симптом "кашель" достоверно чаще встречается у больных раком легких')

несбалансированное соотношениие составляет: 1.8341003714887104e-05 или 0.0%
статистически значимо, так как 1.8341003714887104e-05 <= 0.05
симптом "кашель" достоверно чаще встречается у больных раком легких


In [28]:
data.groupby('LUNG_CANCER')['SHORTNESS_OF_BREATH'].value_counts()

LUNG_CANCER  SHORTNESS_OF_BREATH
1            2                       22
             1                       17
2            2                      176
             1                       94
Name: SHORTNESS_OF_BREATH, dtype: int64

In [29]:
SHORTNESS_OF_BREATH = [[17, 94],[22, 176]]
oddsratio, pvalue = fisher_exact(SHORTNESS_OF_BREATH, alternative='two-sided')
print(f'несбалансированное соотношениие составляет: {pvalue} или {round(pvalue * 100, 2)}%')
if pvalue > 0.05:
    print(f'статистически незначимо, так как {pvalue} > 0.05')
else:
    print(f'статистически значимо, так как {pvalue} <= 0.05')
    print(f'симптом "одышка" достоверно чаще встречается у больных раком легких')

несбалансированное соотношениие составляет: 0.2897420813348365 или 28.97%
статистически незначимо, так как 0.2897420813348365 > 0.05


In [30]:
data.groupby('LUNG_CANCER')['SWALLOWING_DIFFICULTY'].value_counts()

LUNG_CANCER  SWALLOWING_DIFFICULTY
1            1                         34
             2                          5
2            2                        140
             1                        130
Name: SWALLOWING_DIFFICULTY, dtype: int64

In [31]:
SHORTNESS_OF_BREATH = [[34, 130],[5, 140]]
oddsratio, pvalue = fisher_exact(SHORTNESS_OF_BREATH, alternative='two-sided')
print(f'несбалансированное соотношениие составляет: {pvalue} или {round(pvalue * 100, 2)}%')
if pvalue > 0.05:
    print(f'статистически незначимо, так как {pvalue} > 0.05')
else:
    print(f'статистически значимо, так как {pvalue} <= 0.05')
    print(f'симптом "затрудненное глотание" достоверно чаще встречается у больных раком легких')

несбалансированное соотношениие составляет: 3.6496552815358956e-06 или 0.0%
статистически значимо, так как 3.6496552815358956e-06 <= 0.05
симптом "затрудненное глотание" достоверно чаще встречается у больных раком легких


In [32]:
data.groupby('LUNG_CANCER')['CHEST_PAIN'].value_counts()

LUNG_CANCER  CHEST_PAIN
1            1              27
             2              12
2            2             160
             1             110
Name: CHEST_PAIN, dtype: int64

In [33]:
CHEST_PAIN = [[27, 110],[12, 160]]
oddsratio, pvalue = fisher_exact(CHEST_PAIN)
print(f'несбалансированное соотношениие составляет: {pvalue} или {round(pvalue * 100, 2)}%')
if pvalue > 0.05:
    print(f'статистически незначимо, так как {pvalue} > 0.05')
else:
    print(f'статистически значимо, так как {pvalue} <= 0.05')
    print(f'симптом "боли в груди" достоверно чаще встречается у больных раком легких')

несбалансированное соотношениие составляет: 0.000969289691720042 или 0.1%
статистически значимо, так как 0.000969289691720042 <= 0.05
симптом "боли в груди" достоверно чаще встречается у больных раком легких


#### Вывод: Таким образом, все указанные симптомы, за исключением одышки (p = 0.29), встречаются достоверно чаще у больных раком легких при α = 0,05